# Project 3 - Pacman

# Members:
RA 183374 - Helena Steck

RA 187251 - Tainá Turella C. dos Santos

## Introduction
During Project 3 - Pacman the group was required to apply its knowledge around genetic algorithms and reinforcement learning to teach Mr. Pacman to defeat the ghosts while he feeds himself in a maze.
This project was harder than the previous projects because we needed to comprehend how the codified game works and then implement the algorithms in a way that they could interact with each other without friction/issues.

## Code & Difficulties
We used the pacman implementation that is available at [Berkeley](https://inst.eecs.berkeley.edu/~cs188/sp19/assets/files/search.zip "Search.zip") and over this implementation we managed to train our model using reinforcement learning.
We were not able to comprehend how to implement the theory learned about genetic learning, therefore you won't be able to see information about genetic training in this report and we won't be able to make comparisons between the 2 models.

## Reinforcement Learning
The definition of Reinforcement Learning available in [Wikipedia](https://en.wikipedia.org/wiki/Reinforcement_learning "Reinforcement learning") is:
> *Reinforcement learning (RL) is an area of machine learning concerned with how intelligent agents ought to take actions in an environment in order to maximize the notion of cumulative reward.Reinforcement learning is one of three basic machine learning paradigms, alongside supervised learning and unsupervised learning.
Reinforcement learning differs from supervised learning in not needing labelled input/output pairs be presented, and in not needing sub-optimal actions to be explicitly corrected. Instead the focus is on finding a balance between exploration (of uncharted territory) and exploitation (of current knowledge).
The environment is typically stated in the form of a Markov decision process (MDP), because many reinforcement learning algorithms for this context use dynamic programming techniques.The main difference between the classical dynamic programming methods and reinforcement learning algorithms is that the latter do not assume knowledge of an exact mathematical model of the MDP and they target large MDPs where exact methods become infeasible.*

Our implementation will be based on those concepts using Approximate Q-Learning, which is a reinforcement learning algorithm that is based on knowing the value of an action given a particular state. The algorithim calculates the quality of a state-action combination and after each iteration the weigth of each feature is updated and at the end of the training the converged weights are used as params for the Agents.
Using the Aproximate Q-Learning we tried to fit reward function as a linear function that is obtained due to the combination of the features applied and analysed in the model.
One thing that the group believe that should be discussed is the impact that biases can cause in reinforcement learning algorithms. For example, in this project was very tough to teach pacman to eat the ghosts when they are "scared" that happens because in other point it is crucial for pacman to run away from them when they can be harmfull. This is just a silly example, but at the same time made us question somethings during the development.
There are some advantages on using Aproximate Q-Learning, but there are also some disadvantages, such as; the restriction that it applies to the accuracy of learned rewards and it also requires well-known features. It is a more generic algorithm implementation, but its also not perfect.

### Small Grid Training
### Medium Grid Training
### Classic Grid Training

## Disclaimer:
This project was equally divided between the two group members. Therefore we confirm that the methods, the discussions and the report elaboration were created by both of us. We used the code available at [Berkeley](https://inst.eecs.berkeley.edu/~cs188/sp19/assets/files/search.zip "Search.zip") to run pacman and we used their tutorial as a guide on how to implement the methods of Reinforcement Learning.

We did the entire project during calls via Google Meets, using a pair programming method. In that way we could help each other during the development phase.

# Code

We start by importing the necessary files from the pacman game and numpy for helping with math.

In [21]:
%gui tk
from game import Agent, Actions
from pacman import runGames, readCommand
from graphicsDisplay import PacmanGraphics
from ghostAgents import RandomGhost
import numpy as np
import util
import layout
import time
import random

Now we define a base class for our agent which abstracts all the game related things for us and also sets up states for training and running without training

In [22]:
class BaseAgent(Agent):
    def __init__(self, alpha=1.0, epsilon=0.05, gamma=0.8, numTraining=10):
        """
        alpha    - learning rate
        epsilon  - exploration rate
        gamma    - discount factor
        numTraining - number of training episodes
        """
        self.alpha = float(alpha)
        self.epsilon = float(epsilon)
        self.gamma = float(gamma)
        self.numTraining = int(numTraining)

        self.episodesSoFar = 0
        self.accumTrainRewards = 0.0
        self.accumTestRewards = 0.0


    def getLegalActions(self, state):
        """
        Get the actions available for a given
        state. This is what you should use to
        obtain legal actions for a state
        """
        return state.getLegalActions()


    def observeTransition(self, state, action, nextState, deltaReward):
        """
        Inform agent that a transition has
        been observed. This will result in a call to self.update
        on the same arguments
        """
        self.episodeRewards += deltaReward
        self.update(state, action, nextState, deltaReward)


    def startEpisode(self):
        """
        Called when new episode is starting
        """
        self.lastState = None
        self.lastAction = None
        self.episodeRewards = 0.0


    def stopEpisode(self):
        """
        Called when episode is done
        """
        if self.episodesSoFar < self.numTraining:
            self.accumTrainRewards += self.episodeRewards
        else:
            self.accumTestRewards += self.episodeRewards
        self.episodesSoFar += 1
        if self.episodesSoFar >= self.numTraining:
            # Take off the training wheels
            self.epsilon = 0.0  # no exploration
            self.alpha = 0.0  # no learning


    def doAction(self, state, action):
        """
        Called by inherited class when
        an action is taken in a state
        """
        self.lastState = state
        self.lastAction = action

        
    def observationFunction(self, state):
        """
        Called by Pacman game after a new state is generated
        """
        if not self.lastState is None:
            reward = state.getScore() - self.lastState.getScore()
            self.observeTransition(
                self.lastState, self.lastAction, state, reward
            )
        return state

    
    def registerInitialState(self, state):
        """
        Called by Pacman game at the start of a game
        """
        self.startEpisode()
        if self.episodesSoFar == 0:
            print("Beginning %d episodes of Training" % (self.numTraining))


    def final(self, state):
        """
        Called by Pacman game at the terminal state
        """
        deltaReward = state.getScore() - self.lastState.getScore()
        self.observeTransition(
            self.lastState, self.lastAction, state, deltaReward
        )
        self.stopEpisode()

        # Make sure we have this var
        if not "episodeStartTime" in self.__dict__:
            self.episodeStartTime = time.time()
        if not "lastWindowAccumRewards" in self.__dict__:
            self.lastWindowAccumRewards = 0.0
        self.lastWindowAccumRewards += state.getScore()

        NUM_EPS_UPDATE = 100
        if self.episodesSoFar % NUM_EPS_UPDATE == 0:
            print("Reinforcement Learning Status:")
            windowAvg = self.lastWindowAccumRewards / float(NUM_EPS_UPDATE)
            if self.episodesSoFar <= self.numTraining:
                trainAvg = self.accumTrainRewards / float(self.episodesSoFar)
                print(
                    "\tCompleted %d out of %d training episodes"
                    % (self.episodesSoFar, self.numTraining)
                )
                print("\tAverage Rewards over all training: %.2f" % (trainAvg))
            else:
                testAvg = float(self.accumTestRewards) / (
                    self.episodesSoFar - self.numTraining
                )
                print(
                    "\tCompleted %d test episodes"
                    % (self.episodesSoFar - self.numTraining)
                )
                print("\tAverage Rewards over testing: %.2f" % testAvg)
            print(
                "\tAverage Rewards for last %d episodes: %.2f"
                % (NUM_EPS_UPDATE, windowAvg)
            )
            print(
                "\tEpisode took %.2f seconds"
                % (time.time() - self.episodeStartTime)
            )
            self.lastWindowAccumRewards = 0.0
            self.episodeStartTime = time.time()

        if self.episodesSoFar == self.numTraining:
            msg = "Training Done (turning off epsilon and alpha)"
            print("%s\n%s" % (msg, "-" * len(msg)))

Now we implement a function to help us filter the features of our pacman game state. This is important to reduce the complexity in training our Agent by reducing the state space.

In [23]:
def closestFood(pos, food, walls):
    """
    Calculate the distance to the food closest to our pacman by
    also taking in account the walls.

    We do this by doing a fringe search on the 'graph'
    represented by our game board and then for each position
    we check if it has a food, if it has we found our target and
    return the distance.
    """
    fringe = [(pos[0], pos[1], 0)]
    expanded = set()
    while fringe:
        # pop the first pos from the fringe
        pos_x, pos_y, dist = fringe.pop(0)

        # we already visited it: continue
        if (pos_x, pos_y) in expanded:
            continue

        # else: add it to visited locations
        expanded.add((pos_x, pos_y))

        # if we find a food at this location then return the current distance
        if food[pos_x][pos_y]:
            return dist

        # otherwise spread out from the location to its neighbours
        nbrs = Actions.getLegalNeighbors((pos_x, pos_y), walls)
        for nbr_x, nbr_y in nbrs:
            fringe.append((nbr_x, nbr_y, dist + 1))

    # no food found(probably we won the game? if not this will probably bug our Pacman :D)
    return None

In [ ]:
def getFeatures(state, action):
    """
    Returns the following features:
    - bias(always one to ensure training will converge)
    - whether food will be eaten
    - how far away the next food is
    - whether a ghost is one step away
    """
    # extract the grid of food and wall locations and get the ghost locations
    food = state.getFood()
    walls = state.getWalls()
    ghosts = state.getGhostPositions()

    features = util.Counter()

    features["bias"] = 1.0

    # compute the location of pacman after he takes the action
    if action is not None:
        x, y = state.getPacmanPosition()
        dx, dy = Actions.directionToVector(action)
        next_x, next_y = int(x + dx), int(y + dy)
    else:
        x, y = state.getPacmanPosition()
        next_x, next_y = x, y

    # count the number of ghosts 1-step away
    features["#-of-ghosts-1-step-away"] = sum(
        (next_x, next_y) in Actions.getLegalNeighbors(g, walls)
        for g in ghosts
    )

    # if there is no danger of ghosts then add the food feature
    if not features["#-of-ghosts-1-step-away"] and food[next_x][next_y]:
        features["eats-food"] = 1.0

    dist = closestFood((next_x, next_y), food, walls)
    if dist is not None:
        # make the distance a number less than one otherwise the update
        # will diverge wildly
        features["closest-food"] = float(dist) / (
            walls.width * walls.height
        )
    features.divideAll(10.0)
    return features

Now that we have selected which features we want to have in our state, we can implement a class that trains our Agent using an Approximate Q-Learning strategy

In [32]:
class ApproximateQAgent(BaseAgent):
    """
    ApproximateQLearningAgent

    You should only have to overwrite getQValue
    and update.  All other QLearningAgent functions
    should work as is.
    """
    def __init__(self, **args):
        BaseAgent.__init__(self, **args)
        self.weights = util.Counter()


    def getAction(self, state):
        """
        Compute the action to take in the current state.  With
        probability self.epsilon, we should take a random action and
        take the best policy action otherwise.  Note that if there are
        no legal actions, which is the case at the terminal state, you
        should choose None as the action
        """
        # get legal actions 
        action = None
        legalActions = self.getLegalActions(state)
        
        # no action: return
        if len(legalActions) == 0:
            self.doAction(state, action)
            return action

        # explore new action or use Q-Value?
        explore = util.flipCoin(self.epsilon)
        
        # not exploring: choose action with highest Q-Value
        if not explore:
            totalRewards = [self.getQValue(state, a) for a in legalActions]
            action = legalActions[np.argmax(totalRewards)]
        
        # exploring: choose a random action
        else:
            action = random.choice(legalActions)

        # inform base class of action picked
        self.doAction(state, action)
        
        # return picked action
        return action


    def getQValue(self, state, action):
        """
        Should return Q(state,action) = w * featureVector
        where * is the dotProduct operator
        """
        features = getFeatures(state, action)
        return self.weights * features


    def update(self, state, action, nextState, reward):
        """
        Should update weights based on transition
        """
        # get maximum Q-Value after this state change
        legalNextActions = self.getLegalActions(nextState)
        if len(legalNextActions) > 0:
            max_q_value = np.max(
                [self.getQValue(nextState, a) for a in legalNextActions]
            )
        else:
            max_q_value = 0

        # apply the Bellman equation to update the weights
        difference = (reward + self.gamma * max_q_value) - self.getQValue(state, action)

        incremented_features = getFeatures(state, action)
        for k in incremented_features:
            incremented_features[k] *= self.alpha * difference

        self.weights += incremented_features


    def final(self, state):
        "Called at the end of each game."
        # call the super-class final method
        BaseAgent.final(self, state)

        # training finished: print current weights
        if self.episodesSoFar == self.numTraining:
            print(self.weights)

In [34]:
NUM_TRAINING = 1000
runGames(layout.getLayout('smallGrid'),
         ApproximateQAgent(numTraining=NUM_TRAINING),
         [RandomGhost(1), RandomGhost(2), RandomGhost(3), RandomGhost(4)],
         PacmanGraphics(1.0, frameTime = 0.1),
         numGames=NUM_TRAINING,
         numTraining=NUM_TRAINING,
         record=False)

Beginning 1000 episodes of Training
Reinforcement Learning Status:
	Completed 100 out of 1000 training episodes
	Average Rewards over all training: 169.60
	Average Rewards for last 100 episodes: 169.60
	Episode took 0.94 seconds
Reinforcement Learning Status:
	Completed 200 out of 1000 training episodes
	Average Rewards over all training: 134.54
	Average Rewards for last 100 episodes: 99.48
	Episode took 0.85 seconds
Reinforcement Learning Status:
	Completed 300 out of 1000 training episodes
	Average Rewards over all training: 166.28
	Average Rewards for last 100 episodes: 229.76
	Episode took 0.94 seconds
Reinforcement Learning Status:
	Completed 400 out of 1000 training episodes
	Average Rewards over all training: 169.74
	Average Rewards for last 100 episodes: 180.13
	Episode took 0.92 seconds
Reinforcement Learning Status:
	Completed 500 out of 1000 training episodes
	Average Rewards over all training: 171.72
	Average Rewards for last 100 episodes: 179.61
	Episode took 0.95 seconds


[]